In [30]:
#Trims Fastq files of polyA tails for Bowtie Output

file1 = open('D:/ZidLab/SequenceReads/Kyle_7_18_17/WT_5D_FP_S3_L003_R1_001.fastq' , 'r')
output = open('D:/Bowtie/bowtie-1.1.2/WT_5DFP_7_17.fastq' , 'w')
count = 0
count1 = 0
count2 = 0
num = 0
read = 0
for line in file1 :
        num+=1
        if num ==2:  #The file has sequences on the 2nd line, every 4 lines
                temp2 = line.rstrip()
                temp26 = temp2[0:25] #Take the first 26 bps
                tempminus1 = temp26.rstrip('A') #Strip all the A's
                tempminus2 = tempminus1.rstrip('N') #Strip the next base (in case an A was misread
                tempminusA = tempminus2.rstrip('A') #Strip more A's
                Length = len(tempminusA)
                tempTest = tempminusA
                tempTest1 = tempTest[0:-1]
                tempTest2 = tempTest1.rstrip('A')
                LenChange = Length - len(tempTest2) #I'm double checking for extra runs of A's, not sure if its needed.
                if LenChange > 6 :
                        count2 = count2 + 1
                        read = 7
                else:
                        if Length > 17 :
                                if Length > 23:
                                        Sequence = tempminusA[0:23] #Only use the first 23bps for alignment
                                        Length = len(Sequence)      #More likely to have misreads later in the sequence
                                else:
                                        Sequence = tempminusA
                                acount = Sequence.count('A')
                                temp4 = Length - 5
                                if acount < temp4 :
                                        count = count + 1
                                        read = 0
                                else:
                                        read = 7
                        else:
                                read = 7
                
        
        elif num == 4: #This is to get the Quality of the sequence read
                count1+=1
                num = 0
                QualityT = line.rstrip()
                Quality = QualityT[0:Length]
                if read == 7:
                        read = 0
                else:
                        output.write('@r'+str(count)+'/1.'+str(Length)+'\n'+Sequence+'\n'+'+\n'+Quality+'\n')
                
output.close()
print count
print count1
print count2
#Takes txt Solexa Sequence results and outputs them in fastq for
# Took the first 26bp, removed the 3'AAAAs

21475045
28030106
593662


In [4]:
#Converts Bowtie Output to counts per chromosome site
numerals = [
        {'letter': 'M', 'value': 1000},
        {'letter': 'D', 'value': 500},
        {'letter': 'C', 'value': 100},
        {'letter': 'L', 'value': 50},
        {'letter': 'X', 'value': 10},
        {'letter': 'V', 'value': 5},
        {'letter': 'I', 'value': 1},
    ]
def roman_to_arabic(number):
    index_by_letter = {}
    for index in xrange(len(numerals)):
        index_by_letter[numerals[index]['letter']] = index

    result = 0
    previous_value = None
    for letter in reversed(number):
        index = index_by_letter[letter]
        value = numerals[index]['value']
        if (previous_value is None) or (previous_value <= value):
            result += value
        else:
            result -= value
        previous_value = value

    return result



file1 = open('C:/Zid/SolexaDataAnalysis/Kyle/WT1DFP_7_17.map','r') 
file2 = open('C:/Zid/SolexaDataAnalysis/Kyle/output_WT1DFP.fq','r') 
output = open('C:/Zid/SolexaDataAnalysis/Kyle/wt1DFPAlignSortCount.txt','w')
count2 = 0
count3 =0
temp = file1.readline()
line1 = temp.rstrip() 
temp1 = line1.split('\t')
ChrTemp2 = temp1[2]
if ChrTemp2 == "MT":
    Chr = 17
else:
    Chr = roman_to_arabic(ChrTemp2)

Name = temp1[0]
NameLen = Name[-2:]
Orien = temp1[1]
if Orien == '-':
    Posit = (int(temp1[3]))+(int(NameLen))  #didn't subtract 1 adjusting because it is one off
else :
    Posit = (int(temp1[3]))+1 #adjusting because it seems to be one off on the +Strand
    

dct_lst = [{"Chr":Chr,"Posit":Posit,"Orien": Orien}]#"Seq": 'a'}]

for line in file1 :
    count3 +=1
    line1 = line.rstrip() 
    temp1 = line1.split('\t')
    ChrTemp2 = temp1[2]
    if ChrTemp2 == "MT":
        Chr = 17
    else:
        Chr = roman_to_arabic(ChrTemp2)
    Name = temp1[0]
    NameLen = Name[-2:]
    Orien = temp1[1]
    if Orien == '-':
        Posit = (int(temp1[3]))+(int(NameLen))
    elif Orien =='+' :
        Posit = (int(temp1[3]))+1
    temp_lst = [{"Chr":Chr,"Posit":Posit,"Orien": Orien}]#"Seq": 'a'}]#temp1[4]
    dct_lst.extend(temp_lst)

for line in file2 :    
    count3 +=1
    line1 = line.rstrip() 
    temp1 = line1.split('\t')
    ChrTemp2 = temp1[2]
    if ChrTemp2 == "MT":
        Chr = 17
    else:
        Chr = roman_to_arabic(ChrTemp2)
    Name = temp1[0]
    NameLen = Name[-2:]
    Orien = temp1[1]
    if Orien == '-':
        Posit = (int(temp1[3]))+(int(NameLen))
    elif Orien =='+' :
        Posit = (int(temp1[3]))+1
    temp_lst = [{"Chr":Chr,"Posit":Posit,"Orien": Orien}]#"Seq": 'a'}]#temp1[4]
    dct_lst.extend(temp_lst)
    
from operator import itemgetter
dct_lst.sort(key=itemgetter("Posit"))

from operator import itemgetter
dct_lst.sort(key=itemgetter("Chr"))


count = 1
Length = len(dct_lst)
for i in range(len(dct_lst)) :
    Chr = (dct_lst[i]["Chr"])
    Posit = (dct_lst[i]["Posit"])
    Orien = (dct_lst[i]["Orien"])
    #Seq = (dct_lst[i]["Seq"])
    if i == (Length-1):
        output.write(Orien+'\t'+str(Chr)+'\t'+str(count)+'\t'+str(Posit)+'\t'+'\n')#Seq+'\n')
    else:
        OrienT2 = (dct_lst[i+1]["Orien"])
        PositT2 = (dct_lst[i+1]["Posit"])
        if Orien == OrienT2 :
        
            if Posit == PositT2 :
                count = count + 1
            else :
                count2 = count + count2
                output.write(Orien+'\t'+str(Chr)+'\t'+str(count)+'\t'+str(Posit)+'\t'+'\n')#Seq+'\n') #sense count
                count = 1
        else :
            count2 = count + count2
            output.write(Orien+'\t'+str(Chr)+'\t'+str(count)+'\t'+str(Posit)+'\t'+'\n')#Seq+'\n') #sense count
            count = 1
         
output.close()  
print count2 

1617972


In [22]:
#Converts accumulated sorted data to Mochiview format
file1 = open('D:/ZidData/OSheaLab/Solexa/GluTime2/Aligning/AlignedFiles/5P_S2_9p35Align.txt','r')
output = open('D:/ZidData/OSheaLab/MochiView/5Pp35_comb_Mochi.txt', 'w')
output.write('SEQ_NAME\tSTART\tEND\tSTRAND\tCOUNT\n')
#temp7 = file1.readline()
norm = 2.65 # Normalized to total alignable reads/3 million

for line in file1:
    temp = line.rstrip()
    temp1 = temp.split('\t') 
    name = "Chr"+temp1[1]
       
    PositTemp = int(temp1[3])
    Strand = temp1[0]
    if Strand == '+' :
        count = (float(temp1[2]))/norm
        Start = str(PositTemp)
        
    else :
        tempc = (float(temp1[2]))/norm
        count = tempc*(-1)
        Start = str(PositTemp)
        
    output.write(name+'\t'+Start+'\t'+Start+'\t'+Strand+'\t'+str(count)+ '\n')  #antisense count


output.close()
file1.close()

In [24]:
#Converts accumulated sorted data at the genome level to gene features

file1 = open('C:/Zid/SolexaDataAnalysis/EYp35Timecourse/3Pp35CombinedAlignSortCount.txt','r')
output = open('C:/Zid/SolexaDataAnalysis/EYp35Timecourse/3Pp35Feat.txt', 'w')
AlignCountTotal = 0.0

for line in file1:
    temp = line.rstrip()
    temp1 = temp.split('\t') 
    Chr = int(temp1[1])
    Posit = int(temp1[3])
    Posit = Posit + 1
    Orien = temp1[0]
    count = temp1[2]
    AlignCount = 0
    
    #Seq = temp1[5]
    if Chr == 1 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND1.txt','r')
    elif Chr == 2 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND2.txt','r')
    elif Chr == 3 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND3.txt','r')
    elif Chr == 4 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND4.txt','r')
    elif Chr == 5 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND5.txt','r')
    elif Chr == 6 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND6.txt','r')
    elif Chr == 7 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND7.txt','r')
    elif Chr == 8 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND8.txt','r')
    elif Chr == 9 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND9.txt','r')
    elif Chr == 10 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND10.txt','r')
    elif Chr == 11 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND11.txt','r')
    elif Chr == 12 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND12.txt','r')
    elif Chr == 13 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND13.txt','r')
    elif Chr == 14 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND14.txt','r')
    elif Chr == 15 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND15.txt','r')
    elif Chr == 16 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND16.txt','r')
    elif Chr == 17 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND17.txt','r')    
    
    for line2 in file2 :
        temp3 = line2.rstrip()
        temp2 = temp3.split('\t')
        Chr2 = int(temp2[1])
        if Chr == Chr2: #checking if the same chromosome
            if Orien == '-' :  # C orientation
                name = temp2[0]
                Orien2 = temp2[4]
                if Orien2 == '-' :  #also C orientation
                    tStrt2 = int(temp2[3])
                    tEnd2 = int(temp2[2])
                    Strt2 = tStrt2+16
                    End2 = tEnd2+13
                                    
                    if End2 < Posit < Strt2 :
                        AlignCount = AlignCount + 1
                        output.write(name+'\t'+str(tStrt2)+'\t'+str(tEnd2)+'\t'+str(Posit)+'\t'+count+'\n') #sense count
                
                  
            else :          # W orientation
                name = temp2[0]
                Orien2 = temp2[4]
                if Orien2 == '+' :  #also W orientation
                    tStrt2 = int(temp2[2])
                    tEnd2 = int(temp2[3])
                    Strt2 = tStrt2-16
                    End2 = tEnd2-13

                    if Strt2 < Posit < End2 :
                        AlignCount = AlignCount + 1
                        output.write(name+'\t'+str(tStrt2)+'\t'+str(tEnd2)+'\t'+str(Posit)+ '\t'+count+'\n') #sense count
                        AlignCountTotal = AlignCountTotal + float(count)
    file2.close()
print AlignCountTotal
output.close()

960448.0


In [26]:
#Outputs data to excel file with counts per gene
import sys

file1 = open('C:/Zid/SolexaDataAnalysis/EYp35Timecourse/1Pp35Feat.txt','r')
output = open('C:/Zid/SolexaDataAnalysis/EYp35Timecourse/1Pp35Sorted.txt','w')
temp = file1.readline()
line1 = temp.rstrip() 
temp1 = line1.split('\t')
Name = temp1[0]
ChrT = Name[2]
Chr = '1'
dct_lst = [{"name":temp1[0],"Start":temp1[1],"End":"7236","Posit":temp1[3],"Count":temp1[4]}]

for line in file1 :
   temp = line.rstrip()
   temp1 = temp.split('\t')
     
   temp_lst = [{"name":temp1[0],"Start":temp1[1],"End":temp1[2],"Posit":temp1[3],"Count":temp1[4]}]
   dct_lst.extend(temp_lst)
file1.close()

#for line in file2:
  
#   temp = line.rstrip()
#   temp1 = temp.split('\t')  
#   temp_lst = [{"name":temp1[0],"Start":temp1[1],"End":temp1[2],"Posit":temp1[3],"Count":temp1[4],"ASCount": temp1[5]}]
#   dct_lst.extend(temp_lst)

#file2.close()


from operator import itemgetter
dct_lst.sort(key=itemgetter("name"))


for i in range(len(dct_lst)) :
   output.write(dct_lst[i]["name"]+'\t'+dct_lst[i]["Start"]+'\t'+dct_lst[i]["End"]+'\t'+dct_lst[i]["Posit"]+'\t'+dct_lst[i]["Count"]+'\n') #sense count
      
output.close()

file1 = open('C:/Zid/SolexaDataAnalysis/EYp35Timecourse/1Pp35Sorted.txt')
output = open('C:/Zid/SolexaDataAnalysis/EYp35Timecourse/1Pp35Counted.txt','w')
output.write('Name\tStrt\tEnd\tLength\tCount\tStallCount\tCodeCount\t200Count\t200EndCount\n')


tName = 'YAL001C'
tStrt = 147596
tEnd = 151168
tLength = 3573
tCount = 0
Count =0
SCount = 0
CCount = 0
CountS = 0
CountE = 0

for line in file1:
    temp = line.rstrip()
    temp1 = temp.split('\t')
    Name = temp1[0]
   
    if Name[0] == 'Q':
        Count = 0
    elif Name[0] == 'Y':
        Strt = int(temp1[1])
        End = int(temp1[2])
        Orien = Name[6]
        Length = abs(Strt-End)
        Posit = int(temp1[3])
        tCount = int(temp1[4])
        if tName == Name:
            Count = Count + tCount
            if Orien == 'W':
                if Strt+20 > Posit and Strt-16 < Posit:
                    SCount = SCount + tCount
                elif Strt+220 > Posit and Strt-16 < Posit:
                    CountS = CountS + tCount
                elif End - 200 < Posit:
                    CountE = CountE + tCount

            elif Orien == 'C':
                if Strt-20 < Posit and Strt+16 > Posit:
                    SCount = SCount + tCount
                elif Strt-220 < Posit and Strt+16 > Posit:
                    CountS = CountS + tCount
                elif End + 200 > Posit:
                    CountE = CountE + tCount

        else :
            CCount = Count - SCount
            output.write(tName+'\t'+str(tStrt)+'\t'+str(tEnd)+'\t'+str(tLength)+'\t'+str(Count)+'\t'+str(SCount)+'\t'+str(CCount)+'\t'+str(CountS)+'\t'+str(CountE)+'\n')
            Count =0
            SCount = 0
            CCount = 0
            CountS = 0
            CountE = 0
            tName = Name
            Count = Count + tCount
            tStrt = Strt
            tEnd = End
            tLength = Length
            if Orien == 'W':
                if Strt+20 > Posit and Strt-16 < Posit:
                    SCount = SCount + tCount
                elif Strt+220 > Posit and Strt-16 < Posit:
                    CountS = CountS + tCount
                elif End - 200 < Posit:
                    CountE = CountE + tCount

            elif Orien == 'C':
                if Strt-20 < Posit and Strt+16 > Posit:
                    SCount = SCount + tCount
                elif Strt-220 < Posit and Strt+16 > Posit:
                    CountS = CountS + tCount
                elif End + 200 > Posit:
                    CountE = CountE + tCount
output.close()   

In [26]:
#Converts accumulated sorted data at the genome level to gene features

file1 = open('C:/Zid/SolexaDataAnalysis/Anna_7_18_17/RNA3AlignSortCount.txt','r')
output = open('C:/Users/Brian/OneDrive/ZidLab/Data/Solexa/RNA3Feat.txt', 'w')
AlignCountTotal = 0.0

for line in file1:
    temp = line.rstrip()
    temp1 = temp.split('\t') 
    Chr = int(temp1[1])
    Posit = int(temp1[3])
    Posit = Posit + 1
    Orien = temp1[0]
    count = temp1[2]
    AlignCount = 0
    
    #Seq = temp1[5]
    if Chr == 1 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND1.txt','r')
    elif Chr == 2 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND2.txt','r')
    elif Chr == 3 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND3.txt','r')
    elif Chr == 4 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND4.txt','r')
    elif Chr == 5 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND5.txt','r')
    elif Chr == 6 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND6.txt','r')
    elif Chr == 7 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND7.txt','r')
    elif Chr == 8 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND8.txt','r')
    elif Chr == 9 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND9.txt','r')
    elif Chr == 10 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND10.txt','r')
    elif Chr == 11 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND11.txt','r')
    elif Chr == 12 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND12.txt','r')
    elif Chr == 13 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND13.txt','r')
    elif Chr == 14 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND14.txt','r')
    elif Chr == 15 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND15.txt','r')
    elif Chr == 16 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND16.txt','r')
    elif Chr == 17 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND17.txt','r')    
    
    for line2 in file2 :
        temp3 = line2.rstrip()
        temp2 = temp3.split('\t')
        Chr2 = int(temp2[1])
        if Chr == Chr2: #checking if the same chromosome
            if Orien == '-' :  # C orientation
                name = temp2[0]
                Orien2 = temp2[4]
                if Orien2 == '-' :  #also C orientation
                    tStrt2 = int(temp2[3])
                    tEnd2 = int(temp2[2])
                    Strt2 = tStrt2+16
                    End2 = tEnd2+13
                                    
                    if End2 < Posit < Strt2 :
                        AlignCount = AlignCount + 1
                        output.write(name+'\t'+str(tStrt2)+'\t'+str(tEnd2)+'\t'+str(Posit)+'\t'+count+'\n') #sense count
                
                  
            else :          # W orientation
                name = temp2[0]
                Orien2 = temp2[4]
                if Orien2 == '+' :  #also W orientation
                    tStrt2 = int(temp2[2])
                    tEnd2 = int(temp2[3])
                    Strt2 = tStrt2-16
                    End2 = tEnd2-13

                    if Strt2 < Posit < End2 :
                        AlignCount = AlignCount + 1
                        output.write(name+'\t'+str(tStrt2)+'\t'+str(tEnd2)+'\t'+str(Posit)+ '\t'+count+'\n') #sense count
                        AlignCountTotal = AlignCountTotal + float(count)
    file2.close()
print AlignCountTotal
output.close()

820364.0


In [27]:
import pandas as pd
df = pd.read_csv('C:/Users/Brian/OneDrive/ZidLab/Data/Solexa/GenesRibo.txt',sep='\t',header=(0))

df2 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260106_WT_ribo_rep1_counts.txt',sep='\t',header=(0))
df3 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260107_WT_ribo_rep2_counts.txt',sep='\t',header=(0))
df4 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260108_WT_ribo_rep3_counts.txt',sep='\t',header=(0))
df5 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260109_can1_ribo_rep1_counts.txt',sep='\t',header=(0)) 
df6 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260110_can1_ribo_rep2_counts.txt',sep='\t',header=(0))
df7 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260111_can1_ribo_rep3_counts.txt',sep='\t',header=(0))
df8 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260112_WT_mRNA_rep1_counts.txt',sep='\t',header=(0))
df9 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260113_WT_mRNA_rep2_counts.txt',sep='\t',header=(0)) 
df10 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260114_can1_mRNA_rep1_counts.txt',sep='\t',header=(0))
df11 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260115_can1_mRNA_rep2_counts.txt',sep='\t',header=(0))
df12 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260116_rpl22a_ribo_rep1_counts.txt',sep='\t',header=(0))
df13 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260117_rpl22a_ribo_rep2_counts.txt',sep='\t',header=(0)) 
df14 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260118_tor1_ribo_rep1_counts.txt',sep='\t',header=(0))
df15 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260119_tor1_ribo_rep2_counts.txt',sep='\t',header=(0))
df16 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260120_rpl22a_mRNA_counts.txt',sep='\t',header=(0))
df17 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260121_tor1_mRNA_counts.txt',sep='\t',header=(0)) 
new_df = df.merge(df2, how='left', on='Name').merge(df3, how='left', on='Name').merge(df4, how='left', on='Name').merge(df5, how='left', on='Name').merge(df6, how='left', on='Name').merge(df7, how='left', on='Name').merge(df8, how='left', on='Name').merge(df9, how='left', on='Name').merge(df10, how='left', on='Name').merge(df11, how='left', on='Name').merge(df12, how='left', on='Name').merge(df13, how='left', on='Name').merge(df14, how='left', on='Name').merge(df15, how='left', on='Name').merge(df16, how='left', on='Name').merge(df17, how='left', on='Name')

new_df.to_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/dataMerge.txt', sep='\t')


In [14]:
#Combine AlignSortCount Files
#Combine AlignCountSorts  10_8_2014
file1 = open('D:/ZidData/OSheaLab/Solexa/GluTime2/Aligning/AlignedFiles/3P_S1_7p35AlignSortCount.txt') 
file2 = open('D:/ZidData/OSheaLab/Solexa/GluTime2/Aligning/AlignedFiles/3Pp35bAlignSortCount.txt') 

output = open('D:/ZidData/OSheaLab/Solexa/GluTime2/Aligning/AlignedFiles/3Pp35CombinedAlignSortCount.txt','w')

count = 0
temp = file1.readline()
line1 = temp.rstrip() 
temp1 = line1.split('\t')
dct_lst = [{"Orien":temp1[0],"Chr":int(temp1[1]),"Count":int(temp1[2]),"Posit": temp1[3]}]
for line in file1:
    line1 = line.rstrip() 
    temp1 = line1.split('\t')
    temp_lst = [{"Orien":temp1[0],"Chr":int(temp1[1]),"Count":int(temp1[2]),"Posit": int(temp1[3])}]
    dct_lst.extend(temp_lst)
for line in file2:
    line1 = line.rstrip() 
    temp1 = line1.split('\t')
    temp_lst = [{"Orien":temp1[0],"Chr":int(temp1[1]),"Count":int(temp1[2]),"Posit": int(temp1[3])}]
    dct_lst.extend(temp_lst)
    
from operator import itemgetter
dct_lst.sort(key=itemgetter("Posit"))

from operator import itemgetter
dct_lst.sort(key=itemgetter("Chr"))

count2 = 0
CountT = 0
test = 0
Length = len(dct_lst)
for i in range(len(dct_lst)) :
    Chr = (dct_lst[i]["Chr"])
    Posit = (dct_lst[i]["Posit"])
    Orien = (dct_lst[i]["Orien"])
    CountTemp = (dct_lst[i]["Count"])
    
    if i == (Length-1):
        if test == 1:
            output.write(Orien+'\t'+str(Chr)+'\t'+str(CountT)+'\t'+str(Posit)+'\t'+'\n')#Seq+'\n')
            count2 = CountT + count2
        else:
            output.write(Orien+'\t'+str(Chr)+'\t'+str(CountTemp)+'\t'+str(Posit)+'\t'+'\n')
            count2 = CountTemp + count2
    else:
        OrienT2 = (dct_lst[i+1]["Orien"])
        PositT2 = (dct_lst[i+1]["Posit"])
        CountT2 = (dct_lst[i+1]["Count"])
        if Orien == OrienT2 :
        
            if Posit == PositT2 :
                test = 1
                CountT = CountT2 + CountTemp
            else :
                if test == 1:
                    output.write(Orien+'\t'+str(Chr)+'\t'+str(CountT)+'\t'+str(Posit)+'\t'+'\n')#Seq+'\n')
                    count2 = CountT + count2
                    CountT = 0
                else:
                    output.write(Orien+'\t'+str(Chr)+'\t'+str(CountTemp)+'\t'+str(Posit)+'\t'+'\n')
                    count2 = CountTemp + count2
                    CountT = 0
                
                test = 0
        else :
            
            if test == 1:
                output.write(Orien+'\t'+str(Chr)+'\t'+str(CountT)+'\t'+str(Posit)+'\t'+'\n')#Seq+'\n')
                count2 = CountT + count2
                CountT = 0
            else:
                output.write(Orien+'\t'+str(Chr)+'\t'+str(CountTemp)+'\t'+str(Posit)+'\t'+'\n')
                count2 = CountTemp + count2
                CountT = 0
            test = 0
         
output.close()  
print count2 

2360930
